# Chunking Strategies Demo
## Demonstrating 4 Different Chunking Approaches for ArXiv Papers

In [2]:
import sys
print(sys.executable)

/home/adi/Desktop/Projects/agentic-rag/.venv/bin/python


In [4]:
import json
from pathlib import Path
from typing import List, Dict
import re

from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    TokenTextSplitter
)
from langchain_core.documents import Document

/home/adi/Desktop/Projects/agentic-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Sample Paper

In [5]:
# Load first paper for demonstration
import os

# Get the notebook's directory and project root
notebook_dir = Path.cwd()

# Try multiple strategies to find the project root
if (notebook_dir / "raw_data").exists():
    project_root = notebook_dir
elif (notebook_dir.parent / "raw_data").exists():
    project_root = notebook_dir.parent
elif (notebook_dir / "data-ingestion" / "raw_data").exists():
    project_root = notebook_dir / "data-ingestion"
else:
    # Fallback: assume we're in data-ingestion/src
    project_root = Path("/home/adi/Desktop/Projects/agentic-rag/data-ingestion")

text_dir = project_root / "raw_data" / "arxiv" / "text"
metadata_dir = project_root / "raw_data" / "arxiv" / "metadata"

print(f"📂 Working directory: {Path.cwd()}")
print(f"📂 Project root: {project_root}")
print(f"📂 Text directory: {text_dir}")
print(f"📂 Text dir exists: {text_dir.exists()}")
print(f"📂 Files found: {len(list(text_dir.glob('*.txt'))) if text_dir.exists() else 0}\n")

# Get first text file
text_files = sorted(text_dir.glob("*.txt"))
if not text_files:
    raise FileNotFoundError(f"No text files found in {text_dir}")
sample_file = text_files[0]

# Load text
with open(sample_file, 'r', encoding='utf-8') as f:
    paper_text = f.read()

# Load metadata
metadata_file = metadata_dir / f"{sample_file.stem}.json"
with open(metadata_file, 'r') as f:
    metadata = json.load(f)

print(f"📄 Paper: {metadata['title'][:80]}...")
print(f"📌 ArXiv ID: {metadata['arxiv_id']}")
print(f"📁 Category: {metadata['primary_category']}")
print(f"📏 Text Length: {len(paper_text):,} characters")
print(f"📖 Pages: {metadata.get('num_pages', 'N/A')}")

📂 Working directory: /home/adi/Desktop/Projects/agentic-rag/data-ingestion
📂 Project root: /home/adi/Desktop/Projects/agentic-rag/data-ingestion
📂 Text directory: /home/adi/Desktop/Projects/agentic-rag/data-ingestion/raw_data/arxiv/text
📂 Text dir exists: True
📂 Files found: 100

📄 Paper: Amortized Inference of Neuron Parameters on Analog Neuromorphic Hardware...
📌 ArXiv ID: 2602.10763v2
📁 Category: cs.NE
📏 Text Length: 37,293 characters
📖 Pages: 8


In [6]:
# Preview first 500 characters
print("\n📖 Paper Preview:")
print("=" * 80)
print(paper_text[:500])
print("...")
print("=" * 80)


📖 Paper Preview:
Amortized Inference of Neuron Parameters on
Analog Neuromorphic Hardware
Jakob Kaiser∗†, Eric Müller‡ and Johannes Schemmel†
†Institute of Computer Engineering, Heidelberg University, Germany
‡Kirchhoff Institute for Physics, Heidelberg University, Germany
∗jakob.kaiser@kip.uni-heidelberg.de
This article has been accepted for presentation at the Neuro Inspired Computational Elements Conference 2026 and will appear in the conference proceedings.
©2026 IEEE. Personal use of this material is permit
...


## Strategy 1: Recursive Character Splitting (Baseline)

In [7]:
# Recursive character splitter
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

recursive_chunks = recursive_splitter.split_text(paper_text)

print(f"🔹 Recursive Character Splitting")
print(f"   Chunk Size: 1000 chars")
print(f"   Overlap: 200 chars")
print(f"   Total Chunks: {len(recursive_chunks)}")
print(f"\n📊 Chunk Sizes:")
print(f"   Min: {min(len(c) for c in recursive_chunks)} chars")
print(f"   Max: {max(len(c) for c in recursive_chunks)} chars")
print(f"   Avg: {sum(len(c) for c in recursive_chunks) // len(recursive_chunks)} chars")

🔹 Recursive Character Splitting
   Chunk Size: 1000 chars
   Overlap: 200 chars
   Total Chunks: 49

📊 Chunk Sizes:
   Min: 343 chars
   Max: 999 chars
   Avg: 906 chars


In [8]:
# Show first 3 chunks
print("\n📑 First 3 Chunks:")
print("=" * 80)
for i, chunk in enumerate(recursive_chunks[:3], 1):
    print(f"\n--- Chunk {i} ({len(chunk)} chars) ---")
    print(chunk[:300] + "..." if len(chunk) > 300 else chunk)
    print()


📑 First 3 Chunks:

--- Chunk 1 (951 chars) ---
Amortized Inference of Neuron Parameters on
Analog Neuromorphic Hardware
Jakob Kaiser∗†, Eric Müller‡ and Johannes Schemmel†
†Institute of Computer Engineering, Heidelberg University, Germany
‡Kirchhoff Institute for Physics, Heidelberg University, Germany
∗jakob.kaiser@kip.uni-heidelberg.de
This ar...


--- Chunk 2 (952 chars) ---
reuse of any copyrighted component of this work in other works.
Abstract—Our work utilized a non-sequential simulation-based
inference algorithm to provide an amortized neural density
estimator, which approximates the posterior distribution for seven
parameters of the adaptive exponential integrate-...


--- Chunk 3 (975 chars) ---
a more focused posterior and generated posterior predictive traces
that accurately captured the membrane potential dynamics. When
using handcrafted summary statistics, posterior predictive traces
match the included features but show deviations in the exact
dynamics. The posteriors sh

## Strategy 2: Token-Based Chunking

In [9]:
# Token-based splitter
token_splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=50
)

token_chunks = token_splitter.split_text(paper_text)

print(f"🔹 Token-Based Chunking")
print(f"   Max Tokens: 512")
print(f"   Overlap: 50 tokens")
print(f"   Total Chunks: {len(token_chunks)}")
print(f"\n📊 Chunk Sizes (in characters):")
print(f"   Min: {min(len(c) for c in token_chunks)} chars")
print(f"   Max: {max(len(c) for c in token_chunks)} chars")
print(f"   Avg: {sum(len(c) for c in token_chunks) // len(token_chunks)} chars")

🔹 Token-Based Chunking
   Max Tokens: 512
   Overlap: 50 tokens
   Total Chunks: 21

📊 Chunk Sizes (in characters):
   Min: 209 chars
   Max: 2528 chars
   Avg: 1970 chars


In [10]:
# Show first 2 token chunks
print("\n📑 First 2 Token Chunks:")
print("=" * 80)
for i, chunk in enumerate(token_chunks[:2], 1):
    print(f"\n--- Token Chunk {i} ({len(chunk)} chars) ---")
    print(chunk[:300] + "..." if len(chunk) > 300 else chunk)
    print()


📑 First 2 Token Chunks:

--- Token Chunk 1 (2404 chars) ---
Amortized Inference of Neuron Parameters on
Analog Neuromorphic Hardware
Jakob Kaiser∗†, Eric Müller‡ and Johannes Schemmel†
†Institute of Computer Engineering, Heidelberg University, Germany
‡Kirchhoff Institute for Physics, Heidelberg University, Germany
∗jakob.kaiser@kip.uni-heidelberg.de
This ar...


--- Token Chunk 2 (2308 chars) ---
 BayesFlow
I. INTRODUCTION
Identifying suitable models to replicate physical observations
is often a laborious process. Even when a model is available,
determining an effective parameterization can be challenging.
Various methods have been used in neuroscience to achieve
appropriate model parameteri...



## Strategy 3: Section-Based Chunking

In [11]:
# Section detection
section_patterns = [
    r'\bAbstract\b',
    r'\b(?:1\.?\s+)?Introduction\b',
    r'\b(?:\d+\.?\s+)?Related\s+Work\b',
    r'\b(?:\d+\.?\s+)?(?:Methodology|Methods|Approach)\b',
    r'\b(?:\d+\.?\s+)?(?:Experiments|Experimental\s+Results)\b',
    r'\b(?:\d+\.?\s+)?Results\b',
    r'\b(?:\d+\.?\s+)?Discussion\b',
    r'\b(?:\d+\.?\s+)?Conclusion\b',
    r'\b(?:References|Bibliography)\b'
]

def extract_sections(text):
    sections = []
    section_positions = []
    
    for pattern in section_patterns:
        for match in re.finditer(pattern, text, re.IGNORECASE):
            section_name = match.group(0).strip()
            section_positions.append((match.start(), section_name))
    
    section_positions.sort()
    
    for i, (start_pos, section_name) in enumerate(section_positions):
        if i + 1 < len(section_positions):
            end_pos = section_positions[i + 1][0]
        else:
            end_pos = len(text)
        
        section_text = text[start_pos:end_pos].strip()
        section_text = re.sub(section_name, '', section_text, count=1).strip()
        
        if section_text:
            sections.append((section_name, section_text))
    
    if not sections:
        sections.append(("Full Text", text))
    
    return sections

sections = extract_sections(paper_text)

print(f"🔹 Section-Based Chunking")
print(f"   Sections Found: {len(sections)}")
print(f"\n📑 Section Breakdown:")
for section_name, section_text in sections:
    print(f"   • {section_name}: {len(section_text):,} chars")

🔹 Section-Based Chunking
   Sections Found: 28

📑 Section Breakdown:
   • Abstract: 1,153 chars
   • results: 181 chars
   • INTRODUCTION: 194 chars
   • methods: 84 chars
   • methods: 115 chars
   • methods: 83 chars
   • methods: 185 chars
   • methods: 3,680 chars
   • results: 1,129 chars
   • METHODS: 3,910 chars
   • experiments: 72 chars
   • approach: 1,027 chars
   • RESULTS: 329 chars
   • results: 713 chars
   • experiments: 4,706 chars
   • experiments: 2,296 chars
   • experiments: 2,923 chars
   • DISCUSSION: 5,562 chars
   • results: 650 chars
   • Conclusion: 409 chars
   • results: 228 chars
   • methods: 584 chars
   • approach: 1,060 chars
   • methodology: 81 chars
   • methodology: 58 chars
   • methodology: 97 chars
   • REFERENCES: 325 chars
   • Methods: 4,337 chars


In [12]:
# Create chunks from sections (split large sections)
section_chunks = []
for section_name, section_text in sections:
    if len(section_text) > 2000:
        # Split large sections
        subsplitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        subchunks = subsplitter.split_text(section_text)
        for subchunk in subchunks:
            section_chunks.append((section_name, subchunk))
    else:
        section_chunks.append((section_name, section_text))

print(f"\n📊 Total Section-Based Chunks: {len(section_chunks)}")
print("\n📑 Sample Section Chunks:")
print("=" * 80)
for i, (section, chunk) in enumerate(section_chunks[:3], 1):
    print(f"\n--- Chunk {i}: {section} ({len(chunk)} chars) ---")
    print(chunk[:250] + "..." if len(chunk) > 250 else chunk)
    print()


📊 Total Section-Based Chunks: 58

📑 Sample Section Chunks:

--- Chunk 1: Abstract (1153 chars) ---
—Our work utilized a non-sequential simulation-based
inference algorithm to provide an amortized neural density
estimator, which approximates the posterior distribution for seven
parameters of the adaptive exponential integrate-and-fire neuron
model ...


--- Chunk 2: results (181 chars) ---
validate amortized simulation-based
inference as a tool for parameterizing analog neuron circuits.
Index Terms—Neuromorphic Computing, Simulation-Based
Inference, AdEx, BayesFlow
I.


--- Chunk 3: INTRODUCTION (194 chars) ---
Identifying suitable models to replicate physical observations
is often a laborious process. Even when a model is available,
determining an effective parameterization can be challenging.
Various



## Strategy 4: Hybrid Chunking (Recommended)

In [13]:
# Hybrid approach: section-aware + smart splitting
hybrid_chunks = []

for section_name, section_text in sections:
    if len(section_text) < 100:
        continue
    
    # Keep short sections whole
    if section_name.lower() in ['abstract', 'conclusion'] or len(section_text) < 1500:
        hybrid_chunks.append({
            'section': section_name,
            'content': section_text,
            'strategy': 'whole_section'
        })
    else:
        # Smart split for larger sections
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=800,
            chunk_overlap=150,
            separators=["\n\n", "\n", ". ", " ", ""]
        )
        chunks = splitter.split_text(section_text)
        for chunk in chunks:
            hybrid_chunks.append({
                'section': section_name,
                'content': chunk,
                'strategy': 'smart_split'
            })

print(f"🔹 Hybrid Chunking")
print(f"   Total Chunks: {len(hybrid_chunks)}")
print(f"\n📊 Strategy Distribution:")
whole_count = sum(1 for c in hybrid_chunks if c['strategy'] == 'whole_section')
split_count = sum(1 for c in hybrid_chunks if c['strategy'] == 'smart_split')
print(f"   • Whole Sections: {whole_count}")
print(f"   • Smart Split: {split_count}")

🔹 Hybrid Chunking
   Total Chunks: 61

📊 Strategy Distribution:
   • Whole Sections: 15
   • Smart Split: 46


In [14]:
# Show hybrid chunks with metadata
print("\n📑 Sample Hybrid Chunks:")
print("=" * 80)
for i, chunk_data in enumerate(hybrid_chunks[:3], 1):
    print(f"\n--- Hybrid Chunk {i} ---")
    print(f"Section: {chunk_data['section']}")
    print(f"Strategy: {chunk_data['strategy']}")
    print(f"Length: {len(chunk_data['content'])} chars")
    print(f"\nContent Preview:")
    content = chunk_data['content']
    print(content[:300] + "..." if len(content) > 300 else content)
    print()


📑 Sample Hybrid Chunks:

--- Hybrid Chunk 1 ---
Section: Abstract
Strategy: whole_section
Length: 1153 chars

Content Preview:
—Our work utilized a non-sequential simulation-based
inference algorithm to provide an amortized neural density
estimator, which approximates the posterior distribution for seven
parameters of the adaptive exponential integrate-and-fire neuron
model of the analog neuromorphic BrainScaleS-2 substrate...


--- Hybrid Chunk 2 ---
Section: results
Strategy: whole_section
Length: 181 chars

Content Preview:
validate amortized simulation-based
inference as a tool for parameterizing analog neuron circuits.
Index Terms—Neuromorphic Computing, Simulation-Based
Inference, AdEx, BayesFlow
I.


--- Hybrid Chunk 3 ---
Section: INTRODUCTION
Strategy: whole_section
Length: 194 chars

Content Preview:
Identifying suitable models to replicate physical observations
is often a laborious process. Even when a model is available,
determining an effective parameterization can be cha

## Comparison Summary

In [ ]:
import pandas as pd

# Create comparison table
comparison = pd.DataFrame([
    {
        'Strategy': 'Recursive Character',
        'Total Chunks': len(recursive_chunks),
        'Avg Size (chars)': sum(len(c) for c in recursive_chunks) // len(recursive_chunks),
    },
    {
        'Strategy': 'Token-Based',
        'Total Chunks': len(token_chunks),
        'Avg Size (chars)': sum(len(c) for c in token_chunks) // len(token_chunks),
    },
    {
        'Strategy': 'Section-Based',
        'Total Chunks': len(section_chunks),
        'Avg Size (chars)': sum(len(c[1]) for c in section_chunks) // len(section_chunks),
    },
    {
        'Strategy': 'Hybrid (Recommended)',
        'Total Chunks': len(hybrid_chunks),
        'Avg Size (chars)': sum(len(c['content']) for c in hybrid_chunks) // len(hybrid_chunks),
    }
])

print("\n📊 CHUNKING STRATEGIES COMPARISON")
print("=" * 100)
print(comparison.to_string(index=False))
print("=" * 100)


📊 CHUNKING STRATEGIES COMPARISON
            Strategy  Total Chunks  Avg Size (chars)                      Best For
 Recursive Character            49               906   General retrieval, baseline
         Token-Based            21              1970 Direct LLM input (512 tokens)
       Section-Based            58               694 Structured queries by section
Hybrid (Recommended)            61               643   Best overall, section-aware


## Load Pre-computed Chunks from Pipeline

In [16]:
# Load pre-computed chunks from pipeline
chunked_dir = Path("../processed/chunked")

print("📁 Available Chunked Data from Pipeline:")
print("="*80)

for strategy in ['recursive', 'token_based', 'section_based', 'hybrid']:
    strategy_dir = chunked_dir / strategy
    if strategy_dir.exists():
        chunk_files = list(strategy_dir.glob("*.json"))
        print(f"\n🔹 {strategy.replace('_', ' ').title()}")
        print(f"   Papers Chunked: {len(chunk_files)}")
        
        # Load one example
        if chunk_files:
            with open(chunk_files[0], 'r') as f:
                chunks = json.load(f)
            print(f"   Chunks per paper (sample): {len(chunks)}")
            print(f"   First chunk preview: {chunks[0]['content'][:100]}...")

📁 Available Chunked Data from Pipeline:


## Statistics Across All Papers

In [17]:
# Load summary statistics
with open('../logs/chunking_summary.json', 'r') as f:
    summary = json.load(f)

print("\n📊 COMPLETE COLLECTION STATISTICS")
print("="*80)
print(f"\nTotal Papers Processed: {summary['papers_processed']}")
print(f"\nChunks Created:")
print(f"  • Recursive Character: {summary['recursive_chunks']:,}")
print(f"  • Token-Based: {summary['token_chunks']:,}")
print(f"  • Section-Based: {summary['section_chunks']:,}")
print(f"  • Hybrid: {summary['hybrid_chunks']:,}")
print(f"\nTotal Chunks: {sum([summary['recursive_chunks'], summary['token_chunks'], summary['section_chunks'], summary['hybrid_chunks']]):,}")
print("="*80)

FileNotFoundError: [Errno 2] No such file or directory: '../logs/chunking_summary.json'